In [1]:
## https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5

## https://github.com/aboSamoor/polyglot/issues/80
# conda install -c conda-forge pyicu morfessor icu -y && pip install pycld2 polyglot


# polyglot download TASK:ner2
# polyglot download TASK:embeddings2

In [2]:
data_path='../data/'
data_raw_path="../data/raw/"
data_man_path="../data/manipulate/"
model_path="../model/modeling1/"
model_path3="../model/modeling3/"
w2v_path="../data/w2v/"

In [3]:
%matplotlib inline
import re
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.multiclass import OneVsRestClassifier
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import SGDClassifier
import seaborn as sns
import nltk
from string import digits
import ast

import polyglot
from polyglot.text import Text, Word

## Try polyglot and Gensim

In [4]:
# polyglot download pos2.id
# polyglot download embedding.id
# polyglot download morph2.id
# polyglot download sgns2.id

In [5]:
text = Text("halo semua")

In [6]:
text.language.code, text.language.name

('id', 'Indonesian')

In [7]:
## Pos tagging
# text = Text(u"O primeiro uso de desobediência civil em massa ocorreu em setembro de 1906.")
text = Text(u"Saya lahir beberapa tahun lalu Ibu budi memasak Ayam Soeharto di Monas dan KFC")


print("{:<16}{}".format("Word", "POS Tag")+"\n"+"-"*30)
for word, tag in text.pos_tags:
    print(u"{:<16}{:>2}".format(word, tag))


Word            POS Tag
------------------------------
Saya            PRON
lahir           VERB
beberapa        DET
tahun           NOUN
lalu            ADJ
Ibu             PROPN
budi            NOUN
memasak         VERB
Ayam            NOUN
Soeharto        PROPN
di              ADP
Monas           PROPN
dan             CONJ
KFC             PROPN


In [8]:
# NER
text.entities

[I-PER(['Soeharto']), I-LOC(['Monas'])]

In [9]:
for te in ["menggambarkan","mewarnai","menyapu","ambilkan","belikan"]:
    word = Text(te).words[0]
    print(word.morphemes)

Detector is not able to detect the language reliably.


['meng', 'gambar', 'kan']
['me', 'warna', 'i']
['menya', 'pu']
['ambil', 'kan']
['beli', 'kan']


In [10]:
word = Word("nasi", language="id",)
# lontong, sate, soto, mie

In [12]:
word.neighbors

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/numpy/linalg/linalg.py:2390: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


['sup',
 'saus',
 'goreng',
 'bumbu',
 'kuah',
 'roti',
 'sambal',
 'lauk',
 'hidangan',
 'sate']

In [13]:
# https://polyglot.readthedocs.io/en/latest/Embeddings.html
from polyglot.mapping import Embedding
from polyglot.mapping import CaseExpander, DigitExpander
from gensim.models import Word2Vec
from gensim.models.wrappers import FastText
from gensim.models import KeyedVectors


In [14]:
embd0=Embedding.load("/Users/alamhanz/polyglot_data/embeddings2/id/embeddings_pkl.tar.bz2")
# embd1=Embedding.load(w2v_path+"idwiki-latest-pages-articles.xml.bz2")
# embd=Embedding.load(w2v_path+"id/id.bin",)
embd2=Word2Vec.load(w2v_path+"id/id.bin")
embd_f= KeyedVectors.load_word2vec_format(w2v_path+"id_f/id.vec")
# embd_f.load_binary_data(w2v_path+"id_f/id.bin")


In [15]:
embd_f.wv.most_similar(positive="printer",topn=15)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('printed', 0.6384546756744385),
 ('printing', 0.6215873956680298),
 ('scanner', 0.6171170473098755),
 ('print', 0.595815896987915),
 ('pemindai', 0.5931501984596252),
 ('postscript', 0.5672675371170044),
 ('komputer', 0.564993679523468),
 ('laptop', 0.5594507455825806),
 ('desktop', 0.5570290088653564),
 ('proyektor', 0.555558443069458),
 ('device', 0.539400577545166),
 ('server', 0.536881685256958),
 ('workstation', 0.5296144485473633),
 ('konektor', 0.5256128907203674),
 ('pencetakan', 0.5204428434371948)]

In [121]:
embd2.wv.most_similar(positive="buah",topn=10)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bulir', 0.5916192531585693),
 ('buah-buah', 0.575148344039917),
 ('apel', 0.5720770359039307),
 ('biji', 0.5710320472717285),
 ('mangga', 0.5555548667907715),
 ('polong', 0.5521262884140015),
 ('ceri', 0.5376574993133545),
 ('helai', 0.5246707797050476),
 ('bijinya', 0.517110288143158),
 ('tangkai', 0.5115572810173035)]

In [123]:
embd.nearest_neighbors("buah")

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/numpy/linalg/linalg.py:2390: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


['jenis',
 'potongan',
 'daun',
 'koleksi',
 'ekor',
 'warna',
 'dinding',
 'relief',
 'karakter',
 'kulit']

# Get The Data and create our word embedding

In [92]:
# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.W7MeoBMzY3E
# https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
# https://github.com/RaRe-Technologies/gensim/issues/814

In [124]:
def remove_one(X):
    X2=[]
    for c in X:
        if len(c)>=2:
            X2.append(c)
    return X2

In [93]:
from gensim.models import Word2Vec
# define training data

In [94]:
# dtext=pd.read_csv(data_raw_path+"data_text_goshop_2.csv")
# dtag=pd.read_csv(data_raw_path+"data_tag_goshop.csv")

dtext1=pd.read_csv(data_raw_path+"data_text_goshop_2.csv")
dtext2=pd.read_csv(data_raw_path+"data_raw_path_w_label.csv").dropna()
dtext2["real_tags"]=dtext2.real_tags.apply(lambda x : set(ast.literal_eval(x)))
dtag=pd.read_csv(data_raw_path+"data_tag_goshop.csv")

In [110]:
dtext2.sample(15,random_state=100).sort_values("order_no")

,order_no,item_list_cln2,real_tags,num_tags,num_text
714526,248518512,mie bangka komplit satu gapake toge,"{toge, mie}",2,6
135155,259046237,pizza,{pizza},1,1
950635,283917513,kue medan napoleon rasa cheese,{kue},1,5
873501,346976743,nasi ulam x nasi kuning x,"{nasi kuning, nasi}",2,6
927842,360214856,sop buah,{},0,2
256435,417159019,obat jerawat buah krim pemutih i buah,"{krim pemutih, obat jerawat, pemutih, obat, acne}",5,7
360450,437198170,cal stripnexium strip,{},0,3
972036,451994147,roti rasa kacang tanah roti keju,"{roti, kacang}",2,6
475252,456534638,mylanta cair untuk sakit maag,{},0,5
579718,468317272,tasel waran toska sam hitam,{},0,5


In [112]:
dtext1[dtext1.order_no.isin(dtext2.sample(15,random_state=100).order_no)].sort_values("order_no")

,booking_time,order_no,voucher_id,item_list,payment_method_name
714526,2016-11-11 23:21:19,248518512,NaN,2 mie bangka komplit satu gapake toge,CASH
135155,2016-11-27 03:03:18,259046237,NaN,Pizza,GO-PAY
950635,2017-02-05 08:12:45,283917513,NaN,Kue medan napoleon rasa cheese,CASH
873501,2017-08-02 23:50:42,346976743,NaN,"nasi ulam x4, nasi kuning x4",CASH
927842,2017-09-07 05:20:33,360214856,NaN,sop buah,CASH
256435,2017-12-29 01:40:27,417159019,NaN,Obat Jerawat (1buah) dan Krim Pemutih I (1buah),CASH
360450,2018-02-05 13:46:28,437198170,NaN,"Cal 95, 2 stripNexium 1 strip",CASH
972036,2018-03-06 02:42:37,451994147,NaN,roti rasa kacang tanah 2 dan roti keju 2,PARTIAL_PAYMENT
475252,2018-03-13 23:13:37,456534638,NaN,mylanta cair (untuk sakit maag),CASH
579718,2018-04-06 06:35:10,468317272,NaN,tasel waran toska sam hitam,CASH


In [113]:
len(dtext2.item_list_cln2)

1199644

In [135]:
sentence=[]
# for il in dtext2.sample(15,random_state=100).item_list_cln2.tolist()[:10]:
for il in dtext2.item_list_cln2.tolist():
    il2=il.split(" ")
#     sentence.append(il2)
    sentence.append(remove_one(il2))    

In [147]:
len(sentence)

1199644

In [148]:
model_emb = Word2Vec(sentence,size=80,
        window=6,min_count=30,workers=5)

In [149]:
model_emb.train(sentence, total_examples=len(sentence), epochs=10)

(60583568, 75019560)

In [202]:
model_emb.wv.most_similar(positive="susu",topn=20)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('buahsusu', 0.6979660987854004),
 ('bungkussusu', 0.6309378147125244),
 ('pcssusu', 0.6245038509368896),
 ('vanila', 0.6141861081123352),
 ('coklat', 0.5720795392990112),
 ('cokelat', 0.557187557220459),
 ('vanilla', 0.5562208890914917),
 ('stroberi', 0.5549497604370117),
 ('fullcream', 0.5339739322662354),
 ('kgsusu', 0.5332074165344238),
 ('grsusu', 0.5322357416152954),
 ('rasa', 0.5305217504501343),
 ('strawbery', 0.5304673910140991),
 ('plain', 0.5208612680435181),
 ('fortigro', 0.5077904462814331),
 ('lowfat', 0.504540741443634),
 ('astor', 0.4982825815677643),
 ('strawberry', 0.4969242811203003),
 ('kotaksusu', 0.4945349097251892),
 ('skm', 0.4919523000717163)]

In [196]:
embd_f.wv.most_similar(positive="french",topn=20)

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if __name__ == '__main__':
/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('numar', 0.6330020427703857),
 ('numct', 0.6221692562103271),
 ('italian', 0.6202102303504944),
 ('german', 0.608043909072876),
 ('vietnamese', 0.5989323854446411),
 ('numdep', 0.5952830910682678),
 ('numreg', 0.589381754398346),
 ('spanish', 0.5845186710357666),
 ('danish', 0.5767771601676941),
 ('chercher', 0.5730003118515015),
 ('insee', 0.5698051452636719),
 ('tnccnom', 0.5644235610961914),
 ('and', 0.5535695552825928),
 ('russian', 0.5498571991920471),
 ('portuguese', 0.546777606010437),
 ('hungarian', 0.5460720062255859),
 ('belgian', 0.5427613258361816),
 ('japanese', 0.5391103029251099),
 ('romanian', 0.5364443063735962),
 ('cultures', 0.5354399085044861)]

In [201]:
model_emb.wv.similarity(w1="lenovo",w2="handphone")

/Users/alamhanz/anaconda/envs/py36/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.85250556

In [198]:
model_emb.save(w2v_path+'our_vocab.bin')